In [2]:
import pandas as pd
import numpy as np
import time

import pyodbc
import warnings
warnings.filterwarnings('ignore')

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [3]:
base_issn = pd.read_csv('registro_bibliografico_202410151439.csv', delimiter=';')

In [4]:
base_issn = base_issn[base_issn["issn"].str.len() <= 9]
base_issn = base_issn[~base_issn["issn"].str.contains('X')]
list_issn = base_issn['issn'].head(5).to_list()
base_issn.shape, len(list_issn)

((27532, 2), 5)

In [9]:
site = 'https://portal.issn.org/'

# Abrir o navegador
cService = webdriver.ChromeService(executable_path='C:\chromedriver.exe')
navegador= webdriver.Chrome(service = cService)
    
def verifica_issn(issn='0100-1574'):
    navegador.get(site)

    # Pesquisar o valor
    caixa = navegador.find_element(By.XPATH, r'/html/body/div[4]/div/form/div/div/div[1]/input')
    caixa.send_keys(issn)
    navegador.find_element(By.XPATH, r'/html/body/div[4]/div/form/div/div/input').click()

    # Verificar se existe o campo e se constam 2 ou mais registros
    try:
        valor = navegador.find_element(By.XPATH, r'/html/body/div[4]/div/div[2]/div[1]/div/div[3]/div/div/form/div/div[1]/h2').text
    except:
        valor = 'Registro único'

    return valor
    
resultados = []
for issn in list_issn:
    resultado = verifica_issn(issn)
    resultados.append(resultado)

navegador.quit()

In [8]:
base_final = pd.DataFrame({'ISSN':list_issn,
                           'Resultado':resultados})
display(base_final)

,ISSN,Resultado
0,0100-1574,2 results found based on your search
1,1415-8175,2 results found based on your search
2,0101-0405,Registro único
3,1414-4077,2 results found based on your search
4,1677-4833,2 results found based on your search
